In [1]:
#import packages for crawler
import tweepy
from tweepy import OAuthHandler
import requests
import json
import shutil
import os
#import packages for the CNN
from keras.models import model_from_json
import CNN
import twitter_crawler_api

In [2]:
#beg daddy twitter for a scrap of data.

consumer_key = 'KvuDZafQCvKBV6LOkPWMPOfEp'
consumer_secret = '9SZbcgI4kM6ZLkidT5GFK5mf3HS8Pq44bvUn8HlFjomxAsIQj2'
access_token = '1282737152233820162-R1qmQtiPcvw89KOpKx9w1JqbvX1g3c'
access_secret = 'fXR6ip6c1T7TjuQdImsMJbWZ4dvOn8VHBK4cp4njZMoIg'
 
@classmethod
def parse(cls, api, raw):
    status = cls.first_parse(api, raw)
    setattr(status, 'json', json.dumps(raw))
    return status
 
# Status() is the data model for a tweet
tweepy.models.Status.first_parse = tweepy.models.Status.parse
tweepy.models.Status.parse = parse
# User() is the data model for a user profil
tweepy.models.User.first_parse = tweepy.models.User.parse
tweepy.models.User.parse = parse
# You need to do it for all the models you need
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth,wait_on_rate_limit=True)

#username = input("Input the username associated with your twitter account: ")
username = 'megancarrots0'

#get tweets
tweet_id = twitter_crawler_api.get_img(api,username)

In [3]:
#class/index info for classification
#path
data_path = os.path.join('twitter_images','*g')
# load json and create model
json_file = open('model_2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model_2.h5")
loaded_model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam');

In [4]:
#process images, do linear algebra, make cat dir.
data = CNN.process_images(data_path)
is_cat = CNN.final_answer(data,loaded_model)
twitter_crawler_api.get_cats(is_cat,tweet_id,api)